<a href="https://colab.research.google.com/github/SimeonHristov99/CodeEveryDay/blob/main/Dogs_vs_Cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dogs vs. Cats Kaggle Challenge

In [1]:
import os
import zipfile
from glob import glob

from functools import partial

from sklearn.model_selection import train_test_split

import tensorflow as tf
print(tf.__version__)

import numpy as np
import matplotlib.pyplot as plt

2.7.0


In [2]:
config = {
    'kaggle_dir': '../gdrive/MyDrive/kaggle',
    'data_path': '/content/data/train/train',
    'shape': [150, 150, 3]
}

In [3]:
os.chdir('/content')
if not os.path.isdir('data'):
  kaggle_dir = config.get('kaggle_dir', None)

  from google.colab import drive
  drive.mount('/gdrive')
  
  assert kaggle_dir is not None and os.path.isdir(kaggle_dir), 'Kaggle directory not found!'
  assert os.path.isfile(f'{kaggle_dir}/kaggle.json'), 'Kaggle API token not found!'

  !pip install -q kaggle
  os.environ['KAGGLE_CONFIG_DIR'] = kaggle_dir

  # You have to enter in the competition to get access to the data.
  !kaggle competitions download -c dogs-vs-cats -p /content/data/
  assert os.path.isdir('data'), 'ERROR: Could not download the dataset!'

  with zipfile.ZipFile("/content/data/test1.zip","r") as zip_ref:
      zip_ref.extractall("/content/data/test1")
      os.remove("/content/data/test1.zip")

  with zipfile.ZipFile("/content/data/train.zip","r") as zip_ref:
      zip_ref.extractall("/content/data/train")
      os.remove('/content/data/train.zip')

  print('Successfully downloaded the dataset!')
else:
  print('Dataset already downloaded.')

Dataset already downloaded.


## EDA

In [4]:
len(glob('/content/data/train/train/dog*'))

12500

In [5]:
len(glob('/content/data/train/train/cat*'))

12500

In [6]:
len(glob('/content/data/train/train/*'))

25000

Classes are balanced.

## Preprocessing

In [7]:
def train_val_test_split(data_path):
  X = np.array(glob(data_path + '/*'))
  y = np.array([ x.split('.')[0][-3:].lower() for x in X ])

  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
  X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.5, random_state=42)

  return X_train, X_val, X_test, y_train, y_val, y_test

In [8]:
@tf.function
def parse(filenames, resize_to, will_augment):
  image = filenames.get('image', None)
  image = tf.io.read_file(image)
  image = tf.io.decode_jpeg(image, channels=3)
  image = tf.cast(image, tf.float16)
  image = image / 127.5 - 1
  image = tf.image.resize(image, resize_to)

  label = filenames.get('label', None)
  label = 1 if label == 'dog' else 0

  if will_augment:
    return {
        'image': image,
        'label': label,
    }
  
  return image, label

In [9]:
@tf.function
def augment(filenames):
  image = filenames.get('image', None)
  image = tf.image.random_flip_left_right(image)
  # image = tf.image.random_flip_up_down(image)
  # image = tf.image.random_brightness(image, 0.2)
  # image = tf.image.random_contrast(image, 0.5, 2.0)
  # image = tf.image.random_saturation(image, 0.75, 1.25)
  # image = tf.image.random_hue(image, 0.1)
  # image = tf.clip_by_value(image, 0.0, 1.0)  # Keep pixel values between 0 and 1.

  return image, filenames.get('label', None)

In [10]:
def generate_dataset(image_files, label_files, resize_to, shuffle, batch_size, do_augment):
  data_dict = {
      'image': tf.constant(image_files),
      'label': tf.constant(label_files),
  }

  dataset = tf.data.Dataset.from_tensor_slices(data_dict)

  # Parse the files.
  parse_partial_fn = partial(parse, resize_to=resize_to, will_augment=do_augment)
  dataset = dataset.map(parse_partial_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)

  # Cache the parsed files.
  dataset = dataset.cache()

  # Augment the images (if requested).
  if do_augment:
    dataset = dataset.map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)

  # Shuffle the data.
  if shuffle:
    dataset = dataset.shuffle(buffer_size=len(image_files), reshuffle_each_iteration=True)

  # Batch the data.
  dataset = dataset.batch(batch_size, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)

  # Include prefetching.
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return dataset

In [11]:
def get_datasets(resize_to, batch_size, shuffle_X_train, augment_X_train):
  """
  Returns the three datasets (train, validation, test).
  """
  # Get the filepaths to the different sets.
  X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(config.get('data_path', None))

  # Get the datasets of tensors.
  X_train_dataset = generate_dataset(X_train, y_train, resize_to, shuffle=shuffle_X_train, batch_size=batch_size, do_augment=augment_X_train)
  X_val_dataset = generate_dataset(X_val, y_val, resize_to, shuffle=False, batch_size=batch_size, do_augment=False)
  X_test_dataset = generate_dataset(X_test, y_test, resize_to, shuffle=False, batch_size=batch_size, do_augment=False)

  return X_train_dataset, X_val_dataset, X_test_dataset

## Model selection

In [12]:
shape = config.get('shape', None)

In [13]:
X_train_dataset, X_val_dataset, X_test_dataset = get_datasets(resize_to=shape[:2], batch_size=4, shuffle_X_train=True, augment_X_train=True)

base_model = tf.keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=shape,
    include_top=False  # Do not include the ImageNet classifier at the top.
)

base_model.trainable = False

inputs = tf.keras.Input(shape=shape)
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 20,863,529
Trainable params: 2,049
Non-trainable params: 20,861,480
_________________________________________________________________


In [14]:
X_train_dataset, X_val_dataset, X_test_dataset = get_datasets(resize_to=shape[:2], batch_size=4, shuffle_X_train=True, augment_X_train=True)

custom_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same', input_shape=shape),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1)
])

custom_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 conv2d_5 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 37, 37, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 18, 18, 128)      0

## Training

### via a custom model (fast, but not accurate)

In [15]:
custom_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[
                       tf.keras.metrics.BinaryAccuracy()
              ])
custom_model.fit(
    X_train_dataset,
    epochs=2,
    validation_data=X_val_dataset
)

Epoch 1/2
4375/4375 [==============================] - 266s 21ms/step - loss: 1.0373 - binary_accuracy: 0.5547 - val_loss: 1.1685 - val_binary_accuracy: 0.5501
Epoch 2/2
4375/4375 [==============================] - 63s 14ms/step - loss: 0.8216 - binary_accuracy: 0.5765 - val_loss: 1.0545 - val_binary_accuracy: 0.5440


### via transfer learning (takes a lot of time, but you get >= 95% accuracy)

In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[
                       tf.keras.metrics.BinaryAccuracy()
              ])
model.fit(
    X_train_dataset,
    epochs=2,
    validation_data=X_val_dataset
)

Epoch 1/2
4375/4375 [==============================] - 180s 39ms/step - loss: 0.0869 - binary_accuracy: 0.9678 - val_loss: 0.0931 - val_binary_accuracy: 0.9659
Epoch 2/2
4375/4375 [==============================] - 175s 39ms/step - loss: 0.0704 - binary_accuracy: 0.9745 - val_loss: 0.0812 - val_binary_accuracy: 0.9693


In [25]:
model.save_weights('./weights', save_format="h5")

In [37]:
from google.colab import files

files.download('./weights')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
preds = model.predict(X_test_dataset)
preds

array([[ -9.525462 ],
       [-13.823194 ],
       [  1.8771325],
       ...,
       [  7.4840946],
       [  5.8015013],
       [  5.4676533]], dtype=float32)

In [45]:
tf.cast(tf.keras.activations.sigmoid(preds) > 0.5, tf.uint8)

<tf.Tensor: shape=(3750, 1), dtype=uint8, numpy=
array([[0],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=uint8)>